In [1]:
## This time we will do regression on the balance we will predict the value of the income on based on the regression 


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder,OneHotEncoder
import pickle

In [ ]:
data = pd.read_csv("Churn_Modelling.csv")


In [ ]:
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [ ]:
data = data.drop(['RowNumber', 'CustomerId','Surname'],axis=1)

In [ ]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [ ]:
onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [ ]:
data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [ ]:
X = data.drop('EstimatedSalary',axis = 1)
Y = data['EstimatedSalary']
X_train, X_test, Y_train, Y_test =train_test_split(X,Y,random_state=42)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender,file)  
with open('one_hot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo,file)
    
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler,file)
    

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1) ## Here activation function is linear activation function
    ])

d:\Tensorflow Working\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(optimizer='adam', loss = 'mean_absolute_error', metrics=['mae'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

log_dir = "regressionlog/fits/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [ ]:
history = model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_split=0.2,
                    callbacks=[tensorboard_callback, early_stopping])


Epoch 1/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 99442.2344 - mae: 99442.2344 - val_loss: 101327.9062 - val_mae: 101327.9062
Epoch 2/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 99417.7578 - mae: 99417.7578 - val_loss: 100677.3438 - val_mae: 100677.3438
Epoch 3/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 98258.9141 - mae: 98258.9141 - val_loss: 98841.5234 - val_mae: 98841.5234
Epoch 4/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 97912.8594 - mae: 97912.8594 - val_loss: 95452.0547 - val_mae: 95452.0547
Epoch 5/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 94352.3203 - mae: 94352.3203 - val_loss: 90472.7109 - val_mae: 90472.7109
Epoch 6/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 87993.7812 - mae: 87993.7812 - val_loss: 84032.1719 - val_mae: 84032.1719
Epoch 7/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 82036.7109 - mae: 82036.7109 - val_loss: 76929.0703 - val_mae: 76929.0703
Epoch 8/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/st

In [ ]:
%load_ext tensorboard

In [ ]:

%tensorboard --logdir regressionlogs/fits/20250530-115348

Reusing TensorBoard on port 6008 (pid 3400), started 0:00:02 ago. (Use '!kill 3400' to kill it.)

In [ ]:
test_loss,test_mae = model.evaluate(X_test, Y_test)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")

79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 50966.3125 - mae: 50966.3125
Test Loss: 50157.37109375, Test MAE: 50157.37109375


In [ ]:
model.save('regression_model.h5')